In [1]:
import wbdata
import pandas as pd
from tqdm import tqdm
import json
import datetime
import os
import re

In [2]:
country_codes = ['AL', 'AM', 'AO', 'BD', 'BF', 'BJ', 'BO', 'BU', 'CD', 'CF', 'CI', 'CM', 'CO', 'DR', 'EG', 'ET', 'GA', 'GH', 'GN', 'GU', 'GY', 'HN', 'HT', 'IA', 'ID', 'JO', 'KE', 'KH', 'KM', 'LB', 'LS', 'MA', 'MB', 'MD', 'ML', 'MM', 'MW', 'MZ', 'NG', 'NI', 'NM', 'NP', 'PE', 'PH', 'PK', 'RW', 'SL', 'SN', 'SZ', 'TD', 'TG', 'TJ', 'TL', 'TZ', 'UG', 'UZ', 'ZA', 'ZM', 'ZW']

# This dictionary contains the mapping of country codes to country names
country_names_DHS = {
    'AL': 'Albania',
    'AM': 'Armenia',
    'AO': 'Angola',
    'BD': 'Bangladesh',
    'BF': 'Burkina Faso',
    'BJ': 'Benin',
    'BO': 'Bolivia',
    'BU': 'Burundi',
    'CD': 'Democratic Republic of the Congo',
    'CF': 'Central African Republic',
    'CI': "Côte d'Ivoire",
    'CM': 'Cameroon',
    'CO': 'Colombia',
    'DR': 'Dominican Republic',
    'EG': 'Egypt',
    'ET': 'Ethiopia',
    'GA': 'Gabon',
    'GH': 'Ghana',
    'GN': 'Guinea',
    'GU': 'Guatemala',
    'GY': 'Guyana',
    'HN': 'Honduras',
    'HT': 'Haiti',
    'IA': 'Indonesia',
    'ID': 'India',
    'JO': 'Jordan',
    'KE': 'Kenya',
    'KH': 'Cambodia',
    'KM': 'Comoros',
    'LB': 'Lebanon',
    'LS': 'Lesotho',
    'MA': 'Morocco',
    'MB': 'Myanmar',
    'MD': 'Moldova',
    'ML': 'Mali',
    'MM': 'Mauritania',
    'MW': 'Malawi',
    'MZ': 'Mozambique',
    'NG': 'Nigeria',
    'NI': 'Nicaragua',
    'NM': 'Namibia',
    'NP': 'Nepal',
    'PE': 'Peru',
    'PH': 'Philippines',
    'PK': 'Pakistan',
    'RW': 'Rwanda',
    'SL': 'Sierra Leone',
    'SN': 'Senegal',
    'SZ': 'Swaziland',
    'TD': 'Chad',
    'TG': 'Togo',
    'TJ': 'Tajikistan',
    'TL': 'Timor-Leste',
    'TZ': 'Tanzania',
    'UG': 'Uganda',
    'UZ': 'Uzbekistan',
    'ZA': 'South Africa',
    'ZM': 'Zambia',
    'ZW': 'Zimbabwe'
}

country_names_WB= {
    'Albania': 'ALB',
    'Armenia': 'ARM',
    'Angola': 'AGO',
    'Bangladesh': 'BGD',
    'Burkina Faso': 'BFA',
    'Benin': 'BEN',
    'Bolivia': 'BOL',
    'Burundi': 'BDI',
    'Democratic Republic of the Congo': 'COD',
    'Central African Republic': 'CAF',
    "Côte d'Ivoire": 'CIV',
    'Cameroon': 'CMR',
    'Colombia': 'COL',
    'Dominican Republic': 'DOM',
    'Egypt': 'EGY',
    'Ethiopia': 'ETH',
    'Gabon': 'GAB',
    'Ghana': 'GHA',
    'Guinea': 'GIN',
    'Guatemala': 'GTM',
    'Guyana': 'GUY',
    'Honduras': 'HND',
    'Haiti': 'HTI',
    'Indonesia': 'IDN',
    'India': 'IND',
    'Jordan': 'JOR',
    'Kenya': 'KEN',
    'Cambodia': 'KHM',
    'Comoros': 'COM',
    'Lebanon': 'LBN',
    'Lesotho': 'LSO',
    'Morocco': 'MAR',
    'Myanmar': 'MMR',
    'Moldova': 'MDA',
    'Mali': 'MLI',
    'Mauritania': 'MRT',
    'Malawi': 'MWI',
    'Mozambique': 'MOZ',
    'Nigeria': 'NGA',
    'Nicaragua': 'NIC',
    'Namibia': 'NAM',
    'Nepal': 'NPL',
    'Peru': 'PER',
    'Philippines': 'PHL',
    'Pakistan': 'PAK',
    'Rwanda': 'RWA',
    'Sierra Leone': 'SLE',
    'Senegal': 'SEN',
    'Swaziland': 'SWZ',
    'Chad': 'TCD',
    'Togo': 'TGO',
    'Tajikistan': 'TJK',
    'Timor-Leste': 'TLS',
    'Tanzania': 'TZA',
    'Uganda': 'UGA',
    'Uzbekistan': 'UZB',
    'South Africa': 'ZAF',
    'Zambia': 'ZMB',
    'Zimbabwe': 'ZWE'
}


In [3]:
def preprocess(df):
    def extract_year(s):
        match = re.search(r'\d{4}', s)
        return int(match.group()) if match else None

    # Apply the function to create the new 'years' column
    df['DHSYEAR'] = df['DHSID'].apply(lambda x: extract_year(x))

    def extract_country_code(id_value):
        match = re.match(r'^[A-Z]{2,3}', id_value)
        return match.group() if match else None

    # Apply the function to create the new 'Country Code' column
    df['Country_Code'] = df['DHSID'].apply(extract_country_code)

    df = df[['DHSID','DHSYEAR','Country_Code']]

    df.loc[:, 'Country_Code'] = df['Country_Code'].replace('DHS', 'BD')
    return df

In [4]:
# Sample DataFrame with country codes and years
df = pd.read_csv("sample submission.csv")
df = preprocess(df)

# Define the indicators you want to fetch
indicators = {
    # Mean_BMI & Median_BMI
    "AG.PRD.FOOD.XD": "Food production index",
    "AG.PRD.CREL.MT": "Cereal production (metric tons)",
    "AG.YLD.CREL.KG": "Cereal yield (kg per hectare)",
    "SL.AGR.EMPL.ZS": "Employment in agriculture (% of total employment)",
    "AG.CON.FERT.ZS": "Fertilizer consumption (kilograms per hectare of arable land)",
    "SP.RUR.TOTL.ZG": "Rural population growth (annual %)",
    "AG.LND.IRIG.AG.ZS": "Agricultural irrigated land (% of total agricultural land)",
    "AG.LND.AGRI.ZS": "Agricultural land (% of land area)",
    "AG.LND.ARBL.ZS": "Arable land (% of land area)",
    "EG.USE.ELEC.KH.PC": "Electric power consumption (kWh per capita)",
    "EG.USE.PCAP.KG.OE": "Energy use (kg of oil equivalent per capita)",
    "AG.LND.FRST.ZS": "Forest area (% of land area)",
    "SP.POP.GROW": "Population growth (annual %)",
    "SH.STA.MALN.ZS": "Prevalence of underweight, weight for age (% of children under 5)",

    # Unmet_Need_Rate
    "SH.TBS.INCD": "Incidence of tuberculosis (per 100,000 people)",
    "SH.STA.MMRT": "Maternal mortality ratio (modeled estimate, per 100,000 live births)",
    "SH.MED.CMHW.P3": "Community health workers (per 1,000 people)",
    "EN.URB.MCTY.TL.ZS": "Population in urban agglomerations of more than 1 million (% of total population)",

    # Under5_Mortality_Rate
    "SH.DYN.MORT": "Mortality rate, under-5 (per 1,000 live births)",
    "EG.ELC.ACCS.RU.ZS": "Access to electricity, rural (% of rural population)",
    "EN.POP.EL5M.RU.ZS": "Rural population living in areas where elevation is below 5 meters (% of total population)",
    "EN.CLC.MDAT.ZS": "Droughts, floods, extreme temperatures (% of population, average 1990-2009)",
    "AG.LND.EL5M.ZS": "Land area where elevation is below 5 meters (% of total land area)",

    # Skilled_Birth_Attendant_Rate
    "SL.AGR.EMPL.FE.ZS": "Employment in agriculture, female (% of female employment) (modeled ILO estimate)",
    "SP.RUR.TOTL": "Rural population",
    "SH.STA.ANVC.ZS": "Pregnant women receiving prenatal care (%)",
    
    # Stunted_Rate
    "AG.PRD.FOOD.XD": "Food production index",
    "AG.YLD.CREL.KG": "Cereal yield (kg per hectare)",
    "SL.AGR.EMPL.ZS": "Employment in agriculture (% of total employment)",
    "AG.CON.FERT.ZS": "Fertilizer consumption (kilograms per hectare of arable land)",
    "SH.STA.STNT.ZS": "Prevalence of stunting, height for age (% of children under 5)",
    "AG.LND.IRIG.AG.ZS": "Agricultural irrigated land (% of total agricultural land)",
    "AG.LND.AGRI.ZS": "Agricultural land (% of land area)",
    "AG.LND.ARBL.ZS": "Arable land (% of land area)",
    "EN.CLC.MDAT.ZS": "Droughts, floods, extreme temperatures (% of population, average 1990-2009)",
    "AG.LND.FRST.ZS": "Forest area (% of land area)",
    "SH.STA.MALN.ZS": "Prevalence of underweight, weight for age (% of children under 5)",
    
    # Additional Indicators
    "AG.LND.CROP.ZS": "Permanent cropland (% of land area)",
    "AG.LND.EL5M.RU.ZS": "Rural land area where elevation is below 5 meters (% of total land area)",
    "AG.LND.EL5M.UR.ZS": "Urban land area where elevation is below 5 meters (% of total land area)",
    "AG.SRF.TOTL.K2": "Surface area (sq. km)",
    "EN.POP.DNST": "Population density (people per sq. km of land area)",
    "EN.POP.EL5M.UR.ZS": "Urban population living in areas where elevation is below 5 meters (% of total population)",
    "EN.POP.EL5M.ZS": "Population living in areas where elevation is below 5 meters (% of total population)",
    "EN.POP.SLUM.UR.ZS": "Population living in slums (% of urban population)",
    "EN.URB.LCTY.UR.ZS": "Population in the largest city (% of urban population)",
    "EN.URB.MCTY": "Population in urban agglomerations of more than 1 million",
    "SP.POP.TOTL": "Population, total",
    "SP.POP.TOTL.FE.ZS": "Population, female (% of total)",
    "SP.POP.TOTL.MA.ZS": "Population, male (% of total)",
    "SP.RUR.TOTL": "Rural population",
    "SP.RUR.TOTL.ZS": "Rural population (% of total population)"
}

/var/folders/qk/lx707xdn5vx_ghyhq0v4tm540000gn/T/ipykernel_9149/844652245.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Country_Code'] = df['Country_Code'].replace('DHS', 'BD')


In [5]:
# Create a new column for the World Bank's country codes
df['WB_Country_Code'] = df['Country_Code'].map(country_names_DHS).map(country_names_WB)

import pickle

def save_progress(fetched_data, all_data):
    with open('fetched_data.pkl', 'wb') as f:
        pickle.dump(fetched_data, f)
    pd.DataFrame(all_data).to_csv('all_data.csv', index=False)

def load_progress():
    try:
        with open('fetched_data.pkl', 'rb') as f:
            fetched_data = pickle.load(f)
        if os.path.getsize('all_data.csv') > 0:  # Check if the CSV file is not empty
            all_data = pd.read_csv('all_data.csv').to_dict(orient='records')
        else:
            all_data = []
    except (FileNotFoundError, pd.errors.EmptyDataError):  # Added pd.errors.EmptyDataError exception handling
        fetched_data = set()
        all_data = []
    return fetched_data, all_data

def fetch_wb_data_with_progress(df, indicators):
    fetched_data, all_data = load_progress()

    # Unique combinations of years and countries
    unique_years_countries = set(df[['DHSYEAR', 'WB_Country_Code']].itertuples(index=False))

    with tqdm(total=len(unique_years_countries) * len(indicators), desc="Fetching Data") as progress_bar:
        for year, country in unique_years_countries:
            combined_data = {'DHSYEAR': year, 'WB_Country_Code': country}
            for indicator, description in indicators.items():
                if (year, country, indicator) not in fetched_data:
                    try:
                        data_date = (datetime.date(year, 1, 1), datetime.date(year, 12, 31))
                        data = wbdata.get_dataframe({indicator: description}, country=country, data_date=data_date, convert_date=False)
                        combined_data[description] = data.iloc[0, 0] if not data.empty else None
                        fetched_data.add((year, country, indicator))
                    except Exception:
                        pass
                    finally:
                        progress_bar.update(1)
                        save_progress(fetched_data, all_data)  # Save progress after each iteration
                else:
                    progress_bar.update(1)
            
            all_data.append(combined_data)

    data_df = pd.DataFrame(all_data)
    result_df = pd.merge(df, data_df, on=['DHSYEAR', 'WB_Country_Code'], how='left')

    return result_df

# Assuming df and indicators are already defined
result = fetch_wb_data_with_progress(df, indicators)

/var/folders/qk/lx707xdn5vx_ghyhq0v4tm540000gn/T/ipykernel_9149/3233051549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WB_Country_Code'] = df['Country_Code'].map(country_names_DHS).map(country_names_WB)
Fetching Data: 100%|███████████████████████| 3567/3567 [00:09<00:00, 377.16it/s]


In [7]:
result

,DHSID,DHSYEAR,Country_Code,WB_Country_Code,Food production index,Cereal production (metric tons),Cereal yield (kg per hectare),Employment in agriculture (% of total employment),Fertilizer consumption (kilograms per hectare of arable land),Rural population growth (annual %),...,Population density (people per sq. km of land area),Urban population living in areas where elevation is below 5 meters (% of total population),Population living in areas where elevation is below 5 meters (% of total population),Population living in slums (% of urban population),Population in the largest city (% of urban population),Population in urban agglomerations of more than 1 million,"Population, total","Population, female (% of total)","Population, male (% of total)",Rural population (% of total population)
0,AL200800000003,2008,AL,ALB,77.71,608500.00,4081.2,44.65603,77.309738,-2.921578,...,107.566204,NaN,NaN,17.9,26.709050,NaN,2947314.0,50.040062,49.959938,50.009
1,AL200800000005,2008,AL,ALB,77.71,608500.00,4081.2,44.65603,77.309738,-2.921578,...,107.566204,NaN,NaN,17.9,26.709050,NaN,2947314.0,50.040062,49.959938,50.009
2,AL200800000007,2008,AL,ALB,77.71,608500.00,4081.2,44.65603,77.309738,-2.921578,...,107.566204,NaN,NaN,17.9,26.709050,NaN,2947314.0,50.040062,49.959938,50.009
3,AL200800000008,2008,AL,ALB,77.71,608500.00,4081.2,44.65603,77.309738,-2.921578,...,107.566204,NaN,NaN,17.9,26.709050,NaN,2947314.0,50.040062,49.959938,50.009
4,AL200800000009,2008,AL,ALB,77.71,608500.00,4081.2,44.65603,77.309738,-2.921578,...,107.566204,NaN,NaN,17.9,26.709050,NaN,2947314.0,50.040062,49.959938,50.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,ZW201500000382,2015,ZW,ZWE,101.19,800284.41,557.5,66.45557,19.650000,2.312450,...,36.590247,0.0,0.0,NaN,32.715928,1499723.0,14154937.0,52.999890,47.000110,67.615
14996,ZW201500000383,2015,ZW,ZWE,101.19,800284.41,557.5,66.45557,19.650000,2.312450,...,36.590247,0.0,0.0,NaN,32.715928,1499723.0,14154937.0,52.999890,47.000110,67.615
14997,ZW201500000386,2015,ZW,ZWE,101.19,800284.41,557.5,66.45557,19.650000,2.312450,...,36.590247,0.0,0.0,NaN,32.715928,1499723.0,14154937.0,52.999890,47.000110,67.615
14998,ZW201500000390,2015,ZW,ZWE,101.19,800284.41,557.5,66.45557,19.650000,2.312450,...,36.590247,0.0,0.0,NaN,32.715928,1499723.0,14154937.0,52.999890,47.000110,67.615


In [8]:
result.drop_duplicates(subset = "DHSID", inplace=True)

In [9]:
result.to_csv("sample_wbdata.csv",index = False)